# Daily Queries 4

In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, inspect
from IPython.display import display
from pprint import pprint

<div class="alert alert-info"> <b>Student comment v. 1:</b> <br />
    <a href="#q2">updated #2</a>
</div>

<a class="anchor" id="0_toc"></a>
# Table of Contents
***

1. [SQL Engine](#1-engine)
2. [Tables](#2-tables)
3. [Queries](#3-queries)
    1. [What is the diff in percentage between avg yellow cards per game in the cup and champions between df and fw players? <br />(meaning you first need to calculate sum of yellow cards for relevant position's players per game and then divide each position’s total yellow cards by the total of both positions)](#q1)

    2. [Per player calculate the avg percentage of games played.](#q2)

    3. [For each competition return a list of players that didn't get any yellow or red cards](#q3)

    4. [From each position please return the player that played the most in the cup game and had at least one yellow card](#q4)

    5. [What is the average of yellow cards per position? (The share of each position)](#q5)

    6. [Who are the top 3 players with the best assists per minute record? <br />(How often they make an assist – 3 assists in 900 minutes means assist every 300 minutes)](#q6)

    7. [Each game consists of 90 minutes what is the share of each player playing time in the league and cup from his played games? <br />Example: if a player played 1000 minutes and took part in 12 games then his share is 92.59% (1000 / (12*90) )](#q7)

    8. [A forward (fw) is considered to have a good season if he played at least 1500 minutes, <br />took part in at least 18 games and got a minimum of 9 goals + assists. <br />How many forwards like that do we have?](#q8)

<a class="anchor" id="1-engine"></a>
## SQL Engine
***
[back to Table of Contents](#0_toc)

In [2]:
db_name = 'data-analyst-fc-barcelona-20-21-db'

In [3]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': db_name}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})
inspector = inspect(engine)

In [4]:
def read_schema(table_name):
    return pd.DataFrame(inspector.get_columns(table_name)).rename_axis(table_name, axis=1)

In [5]:
def execute_query(q):
    return pd.io.sql.read_sql(q, con=engine)

<a class="anchor" id="2-tables"></a>
## Inspect tables
***
[back to Table of Contents](#0_toc)

In [6]:
tables = inspector.get_table_names()
tables

['squad',
 'games_played',
 'league_stats',
 'champions_stats',
 'cup_stats',
 'supercup_stats']

In [7]:
for table in tables:
    display(read_schema(table))

squad,name,type,nullable,default,autoincrement,comment
0,player_id,INTEGER,True,None,False,None
1,first_name,TEXT,True,None,False,None
2,last_name,TEXT,True,None,False,None
3,position,TEXT,True,None,False,None


games_played,name,type,nullable,default,autoincrement,comment
0,player_id,INTEGER,True,None,False,None
1,league,INTEGER,True,None,False,None
2,champions,INTEGER,True,None,False,None
3,cup,INTEGER,True,None,False,None
4,supercup,INTEGER,True,None,False,None


league_stats,name,type,nullable,default,autoincrement,comment
0,player_id,INTEGER,True,None,False,None
1,league_goals,INTEGER,True,None,False,None
2,league_assists,INTEGER,True,None,False,None
3,league_yellow,INTEGER,True,None,False,None
4,league_red,INTEGER,True,None,False,None
5,league_minutes,INTEGER,True,None,False,None


champions_stats,name,type,nullable,default,autoincrement,comment
0,player_id,INTEGER,True,None,False,None
1,champions_goals,INTEGER,True,None,False,None
2,champions_assists,INTEGER,True,None,False,None
3,champions_yellow,INTEGER,True,None,False,None
4,champions_red,INTEGER,True,None,False,None
5,champions_minutes,INTEGER,True,None,False,None


cup_stats,name,type,nullable,default,autoincrement,comment
0,player_id,INTEGER,True,None,False,None
1,cup_goals,INTEGER,True,None,False,None
2,cup_assists,INTEGER,True,None,False,None
3,cup_yellow,INTEGER,True,None,False,None
4,cup_red,INTEGER,True,None,False,None
5,cup_minutes,INTEGER,True,None,False,None


supercup_stats,name,type,nullable,default,autoincrement,comment
0,player_id,INTEGER,True,None,False,None
1,supercup_goals,INTEGER,True,None,False,None
2,supercup_assists,INTEGER,True,None,False,None
3,supercup_yellow,INTEGER,True,None,False,None
4,supercup_red,INTEGER,True,None,False,None
5,supercup_minutes,INTEGER,True,None,False,None


In [8]:
for table in tables:
    display(execute_query(f'SELECT * FROM {table} LIMIT 1'))

,player_id,first_name,last_name,position
0,1,Marc-André,Ter Stegen,gk


,player_id,league,champions,cup,supercup
0,1,31,5,4,2


,player_id,league_goals,league_assists,league_yellow,league_red,league_minutes
0,1,0,0,1,0,2790


,player_id,champions_goals,champions_assists,champions_yellow,champions_red,champions_minutes
0,1,0,0,0,0,450


,player_id,cup_goals,cup_assists,cup_yellow,cup_red,cup_minutes
0,1,0,0,0,0,420


,player_id,supercup_goals,supercup_assists,supercup_yellow,supercup_red,supercup_minutes
0,1,0,0,0,0,240


<a class="anchor" id="3-queries"></a>
## Queries
***
[back to Table of Contents](#0_toc)

1. [What is the diff in percentage between avg yellow cards per game in the cup and champions between df and fw players? <br />(meaning you first need to calculate sum of yellow cards for relevant position's players per game and then divide each position’s total yellow cards by the total of both positions)](#q1)

2. [Per player calculate the avg percentage of games played.](#q2)

3. [For each competition return a list of players that didn't get any yellow or red cards](#q3)

4. [From each position please return the player that played the most in the cup game and had at least one yellow card](#q4)

5. [What is the average of yellow cards per position? (The share of each position)](#q5)

6. [Who are the top 3 players with the best assists per minute record? <br />(How often they make an assist – 3 assists in 900 minutes means assist every 300 minutes)](#q6)

7. [Each game consists of 90 minutes what is the share of each player playing time in the league and cup from his played games? <br />Example: if a player played 1000 minutes and took part in 12 games then his share is 92.59% (1000 / (12*90) )](#q7)

8. [A forward (fw) is considered to have a good season if he played at least 1500 minutes, <br />took part in at least 18 games and got a minimum of 9 goals + assists. <br />How many forwards like that do we have?](#q8)

<a class="anchor" id="q1"></a>
### 1. What is the diff in percentage between avg yellow cards per game in the cup and champions between df and fw players (meaning you first need to calculate sum of yellow cards for relevant position's players per game and then divide each position’s total yellow cards by the total of both positions)
[up](#3-queries)

In [9]:
# left as documentation (pre-step for the next cell)

execute_query("""
SELECT
    *,
    (cup_yellow + champions_yellow) / (cup_games + champions_games)::real AS avg_yellow
FROM
    (SELECT
        position,
        SUM(cup_yellow) as cup_yellow,
        SUM(games_played.cup) AS cup_games,
        SUM(champions_yellow) AS champions_yellow,
        SUM(games_played.champions) AS champions_games
    FROM
        squad
            LEFT JOIN games_played ON games_played.player_id = squad.player_id
            LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
            LEFT JOIN champions_stats ON champions_stats.player_id = squad.player_id
    WHERE
        position IN ('df', 'fw')
    GROUP BY
        position
    ) AS subq
""")

,position,cup_yellow,cup_games,champions_yellow,champions_games,avg_yellow
0,fw,5,27,2,37,0.109375
1,df,5,32,7,43,0.160000


In [10]:
execute_query("""
SELECT
    position,
    avg_yellow,
    ROUND((avg_yellow * 100 / SUM(avg_yellow) OVER())::numeric, 2) AS avg_yellow_pct
FROM
    (SELECT
        position,
        (SUM(cup_yellow) + SUM(champions_yellow))::real / 
            (SUM(games_played.cup) + SUM(games_played.champions)) AS avg_yellow
    FROM
        squad
            LEFT JOIN games_played ON games_played.player_id = squad.player_id
            LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
            LEFT JOIN champions_stats ON champions_stats.player_id = squad.player_id
    WHERE
        position IN ('df', 'fw')
    GROUP BY
        position
    ) AS subq
ORDER BY
    avg_yellow_pct DESC
""")

,position,avg_yellow,avg_yellow_pct
0,df,0.160000,59.4
1,fw,0.109375,40.6


<div class="alert alert-success" role="alert">
  Great!
</div>

<a class="anchor" id="q2"></a>
### 2. Per player calculate the avg percentage of games played.
[up](#3-queries)

In [11]:
execute_query("""
SELECT
    CONCAT(last_name, ', ', first_name) AS full_name,
    total_games_played,
    ROUND(total_games_played * 100.0 / 54, 2) AS games_played_pct
FROM
    (SELECT
        *,
        league + champions + cup + supercup AS total_games_played
    FROM
        squad
            LEFT JOIN games_played ON games_played.player_id = squad.player_id
    ) AS subq
ORDER BY
    games_played_pct DESC
""")

,full_name,total_games_played,games_played_pct
0,"Lopez, Pedro",52,96.30
1,"Griezmann, Antoine",51,94.44
2,"De Jong, Frenkie",51,94.44
3,"Busquets, Sergio",50,92.59
4,"Alba, Jordi",49,90.74
5,"Messi, Lionel",48,88.89
6,"Lenglet, Clément",48,88.89
7,"Dembélé, Ousmane",44,81.48
8,"Ter Stegen, Marc-André",42,77.78
9,"Trincão, Francisco",42,77.78


<div class="alert alert-warning" role="alert">
  <del>There was supposed to be another field here with amount of games in general but the script that was loaded was the old one so use 54 as the total games to draw precentage
</div>

<div class="alert alert-success" role="alert">
  Great!
</div>

<a class="anchor" id="q3"></a>
### 3. for each competition return a list of players that didn't get any yellow or red cards
[up](#3-queries)

In [12]:
tables

['squad',
 'games_played',
 'league_stats',
 'champions_stats',
 'cup_stats',
 'supercup_stats']

In [13]:
execute_query("""
SELECT column_name
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE table_name LIKE '%%_stats'
          AND column_name ~ '_yellow|_red'
""")

,column_name
0,league_yellow
1,league_red
2,champions_yellow
3,champions_red
4,cup_yellow
5,cup_red
6,supercup_yellow
7,supercup_red


In [14]:
execute_query("""
SELECT
    CONCAT(last_name, ', ', first_name) AS full_name,
    league_yellow + league_red AS league_cards,
    champions_yellow + champions_red AS champions_cards,
    cup_yellow + cup_red AS cup_cards,
    supercup_yellow + supercup_red AS supercup_cards
FROM
    squad
        LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
        LEFT JOIN champions_stats ON champions_stats.player_id = squad.player_id
        LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
        LEFT JOIN supercup_stats ON supercup_stats.player_id = squad.player_id
WHERE
    (league_yellow = 0 AND league_red = 0) AND
    (champions_yellow = 0 AND champions_red = 0) AND
    (cup_yellow = 0 AND cup_red = 0) AND
    (supercup_yellow = 0 AND supercup_red = 0)
""")

,full_name,league_cards,champions_cards,cup_cards,supercup_cards
0,"Murara, Norberto",0,0,0,0
1,"Peña, Iñaki",0,0,0,0
2,"Tenas, Arnau",0,0,0,0
3,"Coutinho, Philippe",0,0,0,0
4,"Fernandes, Matheus",0,0,0,0
5,"De La Fuente, Konard",0,0,0,0
6,"Collado, Álex",0,0,0,0


In [15]:
execute_query("""
(SELECT
    'league' AS competition,
    STRING_AGG(player_id::varchar, ',') AS player_ids
FROM
    league_stats
WHERE league_yellow = 0 AND league_red = 0)
UNION
(SELECT
    'champions',
    STRING_AGG(player_id::varchar, ',')
FROM
    champions_stats
WHERE champions_yellow = 0 AND champions_red = 0)
UNION
(SELECT
    'cup',
    STRING_AGG(player_id::varchar, ',')
FROM
    cup_stats
WHERE cup_yellow = 0 AND cup_red = 0)
UNION
(SELECT
    'supercup',
    STRING_AGG(player_id::varchar, ',')
FROM
    supercup_stats
WHERE supercup_yellow = 0 AND supercup_red = 0)
""")

,competition,player_ids
0,cup,"1,2,3,4,5,6,7,8,9,10,14,15,18,19,20,21,22,23,2..."
1,supercup,"1,2,3,4,5,6,7,8,10,13,14,15,16,17,18,19,20,21,..."
2,champions,"1,2,3,4,6,8,17,18,19,20,21,22,23,24,26,27,29"
3,league,"2,3,4,18,21,23,26"


<div class="alert alert-success" role="alert">
  Great!
</div>

<a class="anchor" id="q4"></a>
### 4. from each position please return the player that played the most in the cup game and had at least one yellow card
[up](#3-queries)

In [16]:
execute_query("""
SELECT
    CONCAT(last_name, ', ', first_name) AS full_name,
    cup_minutes,
    cup_yellow
FROM
    (SELECT
        *,
        MAX(cup_minutes) OVER (PARTITION BY position)
    FROM
        squad
            LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
    WHERE
        cup_yellow >= 1
    ) AS subq
WHERE
    cup_minutes = max
ORDER BY
    cup_minutes DESC,
    cup_yellow DESC
""")

,full_name,cup_minutes,cup_yellow
0,"Messi, Lionel",510,2
1,"Mingueza, Óscar",465,1
2,"Puig, Riqui",162,1


<div class="alert alert-success" role="alert">
  Great!
</div>

<a class="anchor" id="q5"></a>
### 5. What is the average of yellow cards per position? (The share of each position)
[up](#3-queries)

In [17]:
execute_query("""
SELECT
    *,
    ROUND(yellow_cards / SUM(yellow_cards) OVER(), 2) AS yellow_cards_ratio
FROM
    (SELECT
        position,
        SUM(league_yellow + champions_yellow + cup_yellow + supercup_yellow) AS yellow_cards
    FROM
        squad
            LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
            LEFT JOIN champions_stats ON champions_stats.player_id = squad.player_id
            LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
            LEFT JOIN supercup_stats ON supercup_stats.player_id = squad.player_id
    GROUP BY
        position
    ) AS subq
ORDER BY
    yellow_cards_ratio DESC
""")

,position,yellow_cards,yellow_cards_ratio
0,df,46,0.49
1,mf,24,0.26
2,fw,23,0.24
3,gk,1,0.01


<div class="alert alert-success" role="alert">
  Great!
</div>

<a class="anchor" id="q6"></a>
### 6. Who are the top 3 players with the best assists per minute record? (How often they make an assist – 3 assists in 900 minutes means assist every 300 minutes)
[up](#3-queries)

In [18]:
top = 3

execute_query(f"""
SELECT
    CONCAT(last_name, ', ', first_name) AS full_name,
    ROUND(((league_minutes + champions_minutes + cup_minutes + supercup_minutes) 
        / assists::real)::numeric, 2) AS assists_frequency_min
FROM
    (SELECT
        *,
        (league_assists + champions_assists + cup_assists + supercup_assists) AS assists
    FROM
        squad
            LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
            LEFT JOIN champions_stats ON champions_stats.player_id = squad.player_id
            LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
            LEFT JOIN supercup_stats ON supercup_stats.player_id = squad.player_id
    ) AS subq
WHERE
    assists != 0
ORDER BY
    assists_frequency_min
LIMIT {top}
""")

,full_name,assists_frequency_min
0,"Fati, Ansu",149.00
1,"Moriba, Ilaix",224.00
2,"Alba, Jordi",279.73


<div class="alert alert-success" role="alert">
  Great!
</div>

<a class="anchor" id="q7"></a>
### 7. Each game consists of 90 minutes. <br />what is the share of each player playing time in the league and cup from his played games?

Example: if a player played 1000 minutes and took part in 12 games then his share is 92.59% (1000 / (12*90) )  
[up](#3-queries)

In [19]:
execute_query(f"""
SELECT
    CONCAT(last_name, ', ', first_name) AS full_name,
    league_minutes + cup_minutes AS minutes,
    games_played.league + games_played.cup AS games,
    ROUND(
        CASE WHEN (games_played.league + games_played.cup) != 0 THEN
            (league_minutes + cup_minutes) / ((games_played.league + games_played.cup)*90.0)
        ELSE 0 END, 2) AS ratio
FROM
    squad
        LEFT JOIN games_played ON games_played.player_id = squad.player_id
        LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
        LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
ORDER BY
    ratio DESC
""")

,full_name,minutes,games,ratio
0,"Murara, Norberto",840,9,1.04
1,"Ter Stegen, Marc-André",3210,35,1.02
2,"De Jong, Frenkie",3668,42,0.97
3,"Messi, Lionel",3532,41,0.96
4,"Alba, Jordi",3472,40,0.96
5,"Piqué, Gerard",1687,20,0.94
6,"Lenglet, Clément",2911,38,0.85
7,"Griezmann, Antoine",3160,42,0.84
8,"Mingueza, Óscar",2366,32,0.82
9,"Busquets, Sergio",2997,42,0.79


<div class="alert alert-success" role="alert">
  Great!
</div>

<a class="anchor" id="q8"></a>
### 8. A forward (fw) is considered to have a good season if he played at least 1500 minutes , took part in at least 18 games and got a minimum of 9 goals + assists.

How many forwards like that do we have?  
[up](#3-queries)

In [20]:
position = 'fw'

execute_query(f"""
SELECT
    *,
    ROUND((goals + assists) / minutes_played::numeric, 3) AS goals_assists_to_minutes
FROM
    (SELECT
        CONCAT(last_name, ', ', first_name) AS full_name,
        games_played.league + games_played.champions + games_played.cup + games_played.supercup AS games_played,
        league_minutes + champions_minutes + cup_minutes + supercup_minutes AS minutes_played,
        league_goals + champions_goals + cup_goals + supercup_goals AS goals,
        league_assists + champions_assists + cup_assists + supercup_assists AS assists

    FROM
        squad
            LEFT JOIN games_played ON games_played.player_id = squad.player_id
            LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
            LEFT JOIN champions_stats ON champions_stats.player_id = squad.player_id
            LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
            LEFT JOIN supercup_stats ON supercup_stats.player_id = squad.player_id
    WHERE
            squad.position = '{position}'
    ) AS subq
WHERE
    games_played >= 18 AND
    minutes_played >= 1500 AND
    goals + assists >= 9
ORDER BY
    goals_assists_to_minutes DESC
""")

,full_name,games_played,minutes_played,goals,assists,goals_assists_to_minutes
0,"Messi, Lionel",48,4192,38,14,0.012
1,"Griezmann, Antoine",51,3903,20,13,0.008
2,"Dembélé, Ousmane",44,2699,11,5,0.006
3,"Braithwaite, Martin",40,1716,7,4,0.006


There are 4 players matching the description.

<div class="alert alert-success" role="alert">
  Great!
</div>